In [21]:
import os
import time
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

In [22]:
#edit sesuai kebutuhan
vessel_type = 'bulk carrier'
filenum = '1'
kelas = 'NK'

In [23]:
#directory file csv
read_path = 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/input/{}/split_{}.csv'.format(vessel_type,filenum)
write_path = 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/output/{}/{}/{}.csv'.format(kelas,vessel_type,filenum)

In [24]:
# Check if input file exists
if os.path.exists(read_path):
    print(f"CSV read file is found, continuing...")
else:
    print(f"CSV read file is not found")
    print('exiting....')
    
# Check if output file exists and create it if necessary
if os.path.exists(write_path):
    print(f"CSV write file  is found, continuing...")
else:
    print(f"CSV write file is not found, creating one...")
    os.makedirs(os.path.dirname(write_path), exist_ok=True) # Create output directory if it doesn't exist
    with open(write_path, 'w', newline='') as f:
        writer = csv.writer(f)

CSV read file is not found
exiting....
CSV write file  is found, continuing...


In [25]:
def search(IMO):
    driver.get("https://www.classnk.com/register/regships/regships.aspx")

    search_field = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/form/div[3]/table/tbody/tr[2]/td/input')))
    search_field.send_keys(IMO)

    search_button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/form/div[3]/p/a[1]')
    search_button.click()
    
    ship_button = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/div[1]/div[2]/h3/a')))
    ship_button.click()

In [26]:
def newtab():
    # Open a new tab with the desired URL
    driver.execute_script("window.open('https://www.classnk.com/register/regships/one.aspx', '_blank');")

    # Switch to the new tab
    time.sleep(3)
    driver.switch_to.window(driver.window_handles[-1])

In [27]:
def shipdata():
    global LOA, LPP, B, H, T
    global DWT, NT, GT, LT
    global NAME, REGISTER, IMO, vessel_type
    global link
    
    wait = WebDriverWait(driver, 10) # Maximum wait time of 10 seconds
    
    NAME = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[8]/td')))
    REGISTER = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[2]/td')))
    IMO = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[3]/td')))
    vessel_type = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[25]/td')))
    
    #principal dimension
    LOA = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[45]/td')))
    T = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[40]/td')))
    
    dimensions = wait.until(EC.visibility_of_element_located((By.XPATH , '/html/body/div/div/div[1]/div[3]/table/tbody/tr[44]/td')))
    overall = str(dimensions.text)    
    split = overall.split('x')
    LPP, B, H = split
    
    #ship tonnage
    GT = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[31]/td')))
    NT = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[32]/td')))
    DWT = wait.until(EC.visibility_of_element_located((By.XPATH,'/html/body/div/div/div[1]/div[3]/table/tbody/tr[37]/td')))
    
    link = 'https://www.vesselfinder.com/vessels/details/{}'.format(IMO.text)
    
#     print variable names and value
    print('NAME:', NAME.text)
    print('REGISTER:', REGISTER.text)
    print('IMO:', IMO.text)
    print('VESSEL TYPE:', vessel_type.text)
    print('LOA:', LOA.text)
    print('LPP:', LPP)
    print('T:', T.text)
    print('B:', B)
    print('H:', H)
    print('GT:', GT.text)
    print('NT:', NT.text)
    print('DWT:', DWT.text)
    print('LINK:', link)

In [28]:
# Define the fieldnames for the CSV file
fieldnames = ['NAME', 'REGISTER', 'IMO', 'TYPE', 'LOA', 'LPP', 'B', 'H', 'T',  'GT', 'NT', 'DWT', 'Link']

In [29]:
# Set up the webdriver
driver = webdriver.Chrome()

driver.get("https://www.classnk.com/register/regships/regships.aspx")

IMO = '9278923'
search(IMO)
newtab()
shipdata()
print('yahalo')

NAME: LILA 
REGISTER: 032894
IMO: 9278923
VESSEL TYPE: Other than Oil Tanker
LOA: 169.400 x 26.000 x 13.600 
LPP: 169.400 
T: 16.0 
B:  26.000 
H:  13.600 
GT: -- 
NT: -- 
DWT: 4,020 
LINK: https://www.vesselfinder.com/vessels/details/9278923
yahalo


In [20]:
# Set up the webdriver
driver = webdriver.Chrome()

# Open the CSV file for writing
with open(write_path, mode='w', newline='') as details_file:
    writer = csv.DictWriter(details_file, fieldnames=fieldnames)
    writer.writeheader()
    
    # Open the input file and loop through each vessel name
    with open(read_path, newline='') as csvfile:
        reader = csv.reader(csvfile)        
        for row in reader:
            IMO = row[0]
            try:
                search(IMO)
                newtab()
                shipdata()
                print('ketemu IMO :',IMO.text)
            except:
                print('ga ketemu IMO :',IMO)
                continue
                
                           
            details_dict = {'NAME': NAME.text, 'REGISTER': REGISTER.text, 'IMO': IMO.text, 'TYPE':  vessel_type.text, 'LOA': LOA.text, 'LPP': LPP, 'B': B, 'H': H, 'T': T.text, 'GT': GT.text, 'NT': NT.text, 'DWT': DWT.text, 'Link': link}
            writer.writerow(details_dict)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/abinp/Documents/ITS/side projects/new/ship data scrapping/input/bulk carrier/split_1.csv'

In [ ]:
# Set up the webdriver
driver = webdriver.Chrome()

driver.get("https://www.classnk.com/register/regships/regships.aspx")

IMO = '9317456'

time.sleep(5)
search_field = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/form/div[3]/table/tbody/tr[2]/td/input')
search_field.send_keys(IMO)

search_button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/form/div[3]/p/a[1]')
search_button.click()

time.sleep(2)
ship_button = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[3]/div/div[1]/div[2]/h3/a')
ship_button.click()

# Open a new tab with the desired URL
driver.execute_script("window.open('https://www.classnk.com/register/regships/one.aspx', '_blank');")

# Switch to the new tab
driver.switch_to.window(driver.window_handles[-1])

time.sleep(5)
name = driver.find_element(By.XPATH, '/html/body/div/div/div[1]/div[3]/table/tbody/tr[8]/td')
print(name.text)